In [ ]:
import os
os.chdir('../')

# 使用MMRazor对ResNet34进行剪枝

## 回顾MMCls

In [17]:
# Prepare config path
work_dir='./prune_example/'
config_path=f"{work_dir}/configs/"
! mkdir -p $config_path

pretrained_path='https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'

In [18]:
! echo "_base_ = ['mmcls::resnet/resnet34_8xb32_in1k.py']" > $config_path/pretrain.py
! cat $config_path/pretrain.py

# Run config
# ! python ./tools/train.py $config_path/pretrain.py

_base_ = ['mmcls::resnet/resnet34_8xb32_in1k.py']


## 准备剪枝结构搜索config
1. 生成搜索config
2. 运行搜索config
3. 获得搜索出的最优结构

### 1.生成搜索config
我们提供了一个工具可以帮助我们一条命令行生成搜索算法config。改工具需要我们提供两个参数：config路径与checkpoint路径

In [19]:
! python tools/get_search_config.py -h

usage: get_search_config.py [-h] [--flops-min FLOPS_MIN]
                            [--flops-max FLOPS_MAX] [-o O]
                            config checkpoint

positional arguments:
  config
  checkpoint

optional arguments:
  -h, --help            show this help message and exit
  --flops-min FLOPS_MIN
  --flops-max FLOPS_MAX
  -o O


In [ ]:
! python tools/get_search_config.py  $config_path/pretrain.py $pretrained_path -o $config_path/search.py   &> /dev/null 
! cat $config_path/search.py 

### 2. 我们可以直接运行该config从而进行剪枝结构的搜索

In [ ]:
! python ./tools/train.py $config_path/search.py  --work-dir $work_dir/search/

### 3. 获取搜索结果

In [ ]:
!  cat $work_dir/search/best_fix_subnet.json

## 准备剪枝Config

In [ ]:
! python ./tools/get_prune_config.py $config_path/pretrain.py $pretrained_path -o $config_path/prune.py &> /dev/null
! cat $config_path/prune.py

In [ ]:
# change prune config
from mmengine import fileio,Config
pruning_subnet=fileio.load(f'{work_dir}/search/best_fix_subnet.json')
pruning_config=Config.fromfile(f'{config_path}/prune.py')
pruning_config['model']['target_pruning_ratio']=pruning_subnet
pruning_config.dump(f'{config_path}/prune.py')
! cat $config_path/prune.py

In [ ]:
# run your prune config
! python ./tools/train.py $config_path/prune.py --work-dir $work_dir/prune

In [16]:
# ! rm -r $work_dir